# Bivariate matrix maps

Bivariate choropleth maps are created to highlight areas with varying levels of social vulnerability and flood map accuracy. In these maps, two color tones represent the two variables: one for the accuracy of the official flood maps and the other for the SoVI scores. Each color tone is divided into three color intensity levels based on the 25th, 50th, and 75th percentiles of the variable distribution. 

# Install Necessary packages

In [ ]:
# pip install geopandas
# pip install pandas
# pip install matplotlib
# pip install numpy

# Load packages

In [ ]:
import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap
from matplotlib.patches import Rectangle
import matplotlib.gridspec as gridspec

In [ ]:
# ───────────────────────────────────────── CONFIG ─────────────────────────────────────────
BASE_DIR = r"BASE_DIR if user have any"
DATA_DIR = rf"path of shapefile with flood performance metrics of population exposure and SoVI_Score"
OUT_DIR  = rf"{BASE_DIR}/Output/Output_maps"

#Selecting different shapefile, it could be replaced by the shapefile name from DATA_DIR
REGIONS = {
    "Long Bayou":         "Long_floodpop_matrix.shp",
    "Shore Acres":        "Shore_floodpop_matrix.shp",
    "USF St. Petersburg": "StPt_floodpop_matrix.shp",
}
#Selecting fields for different flood performance metrics for tww different flood maps
METRICS = {
    "FEMA": ["Fe100_acc", "Fe100_pre", "Fe100_rec"],
    "FDEM": ["Evac_acc",  "Evac_pre",  "Evac_rec"],
}
#Selecting SOVI_Score
SOVI_FIELD = "SoVI_Score"

#Labeling each map
MAP_LABELS = np.array(list("abcdefghi")).reshape(3, 3)

COLOR_MATRIX = np.array([
    ['#e571b8', '#9c5dac', '#5548a0'],  # High SoVI (row 0 – TOP)
    ['#e9afd7', '#a698c9', '#6483bc'],  # Mid  SoVI (row 1)
    ['#edebf6', '#b0d5e7', '#72bdda'],  # Low  SoVI (row 2 – BOTTOM)
])
CMAP = ListedColormap(COLOR_MATRIX.ravel())

# ──────────────────────────────────────── UTILITIES ───────────────────────────────────────

def ensure_dir(path: str) -> None:
    """Create directory if it does not exist."""
    os.makedirs(path, exist_ok=True)


def classify_series(series: pd.Series, bins: np.ndarray) -> pd.Series:
    """Quantile classification into High(0)/Mid(1)/Low(2)."""
    return pd.cut(series, bins=[-np.inf, *bins, np.inf], labels=[2, 1, 0]).astype(int)


def global_bins(gdf: gpd.GeoDataFrame, field: str) -> np.ndarray:
    """Return 25th and 75th global quantiles for a field."""
    return gdf[field].quantile([0.25, 0.75]).values

# ────────────────────────────────────────── CORE ──────────────────────────────────────────

def build_combined_gdf(region_map: dict) -> dict[str, gpd.GeoDataFrame]:
    """Load regional shapefiles into a dict of GeoDataFrames."""
    return {region: gpd.read_file(os.path.join(DATA_DIR, fname)) for region, fname in region_map.items()}


def draw_bivariate(
    gdfs: dict[str, gpd.GeoDataFrame],
    metric_fields: list[str],
    title: str,
    outfile_stub: str,
    alpha: float = 0.8,
    layout_only: bool = False,
):
    """Render 3×3 grid of bivariate maps and save png/svg."""

    fig = plt.figure(figsize=(12, 10))
    gs = gridspec.GridSpec(3, 3, wspace=0.05, hspace=0.05)
    plt.subplots_adjust(left=0.05, right=0.95, top=0.92, bottom=0.05)

    combined = gpd.GeoDataFrame(pd.concat(gdfs.values(), ignore_index=True))
    sov_bins = global_bins(combined, SOVI_FIELD)
    met_bins = {m: global_bins(combined, m) for m in metric_fields}

    for col, (region, gdf_raw) in enumerate(gdfs.items()):
        for row, metric in enumerate(metric_fields):
            ax = plt.subplot(gs[row, col])
            gdf = gdf_raw.copy()

            gdf["sov_cls"] = classify_series(gdf[SOVI_FIELD], sov_bins)
            gdf["met_cls"] = classify_series(gdf[metric], met_bins[metric])
            gdf["bivar"] = gdf["sov_cls"] * 3 + gdf["met_cls"]

            gdf.plot(ax=ax, column="bivar", cmap=CMAP, linewidth=0.15, edgecolor="white", alpha=alpha)

            xmin, ymin, xmax, ymax = gdf.total_bounds
            ax.set_xlim(xmin + 0.05 * (xmax - xmin), xmax - 0.05 * (xmax - xmin))
            ax.set_ylim(ymin + 0.05 * (ymax - ymin), ymax - 0.05 * (ymax - ymin))
            ax.axis("off")

            if not layout_only:
                ax.set_title(f"({MAP_LABELS[row, col]})", loc="left", fontsize=11)

    if not layout_only:
        legend_ax = fig.add_axes([0.38, 0.01, 0.24, 0.24])
        for r in range(3):
            for c in range(3):
                legend_ax.add_patch(Rectangle((c, r), 1, 1, facecolor=COLOR_MATRIX[r, c]))
        legend_ax.set_xlim(0, 3)
        legend_ax.set_ylim(0, 3)
        legend_ax.invert_yaxis()  # <-- Flip so row 0 (High SoVI) sits at the top
        legend_ax.set_xticks([0.5, 1.5, 2.5])
        legend_ax.set_yticks([0.5, 1.5, 2.5])
        legend_ax.set_xticklabels(["H", "M", "L"])
        legend_ax.set_yticklabels(["H", "M", "L"])
        legend_ax.set_xlabel("Performance")
        legend_ax.set_ylabel("SoVI")
        legend_ax.tick_params(axis='both', length=0)
        legend_ax.set_frame_on(False)

    if title:
        fig.suptitle(title, fontsize=14)

    ensure_dir(OUT_DIR)
    png_path = os.path.join(OUT_DIR, f"{outfile_stub}.png")
    
    # svg file export for further modification in map
    svg_path = os.path.join(OUT_DIR, f"{outfile_stub}.svg")
    fig.savefig(svg_path if layout_only else png_path, dpi=300, transparent=layout_only) 
    plt.close()

# ───────────────────────────────────────── DRIVER ─────────────────────────────────────────
if __name__ == "__main__":
    gdf_dict = build_combined_gdf(REGIONS)

    # FEMA ― full + layout‑only
    draw_bivariate(
        gdf_dict,
        METRICS["FEMA"],
        title="Bivariate SoVI vs FEMA 100‑Year Flood Map Performance",
        outfile_stub="Bivariate_FEMA",
        layout_only=False,
    )
    draw_bivariate(
        gdf_dict,
        METRICS["FEMA"],
        title="",
        outfile_stub="Bivariate_FEMA_layout_only",
        alpha=0.7,
        layout_only=True,
    )

    # FDEM ― full + layout‑only
    draw_bivariate(
        gdf_dict,
        METRICS["FDEM"],
        title="Bivariate SoVI vs FDEM Evacuation Zone Performance",
        outfile_stub="Bivariate_FDEM",
        layout_only=False,
    )
    draw_bivariate(
        gdf_dict,
        METRICS["FDEM"],
        title="",
        outfile_stub="Bivariate_FDEM_layout_only",
        alpha=0.7,
        layout_only=True,
    )